In [1]:
from llama_index.core import SQLDatabase
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///chinook.db")
sql_database = SQLDatabase(engine)

In [2]:
from llama_index.core.query_pipeline import QueryPipeline

In [8]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.tools import QueryEngineTool
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama

Settings.llm = Ollama(model="llama3.2", request_timeout=3000.0)
Settings.embed_model = OptimumEmbedding(folder_name="./minilml6v2_onnx")

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    verbose=True,
)
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query"
    ),
)

In [9]:
from llama_index.core.query_pipeline import QueryPipeline as QP

qp = QP(verbose=True)

In [12]:
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools(
    [sql_tool], 
    llm=Settings.llm, 
    verbose=True,
    max_iterations=30,
)

In [13]:
response = agent.chat("What are some tracks from the artist AC/DC? Limit it to 3")
print(response)

> Running step 41d468c1-d42a-4dc2-b26c-d26e1b1e3f1b. Step input: What are some tracks from the artist AC/DC? Limit it to 3
Thought: The user wants information about a specific artist, AC/DC. I need to use a tool to help me answer the question.
Action: sql_tool
Action Input: {'input': "SELECT title FROM albums WHERE artist = 'AC/DC' LIMIT 3", 'num_beams': 5}
Observation: Error: albums
> Running step d2042b00-54eb-49bf-a7b4-17c23205438a. Step input: None
Thought: The user's tool response indicates that an error occurred. I need to try a different approach.
Action: sql_tool
Action Input: {'input': "SELECT title FROM song IN (SELECT title FROM albums) WHERE artist = 'AC/DC' LIMIT 3", 'num_beams': 5}
Observation: Error: albums
> Running step 73c539ef-2692-4baf-b549-12c782fa82a0. Step input: None
Thought: The user's tool response still indicates an error. I should try a more general approach.
Action: sql_tool
Action Input: {'input': "SELECT title FROM song WHERE artist = 'AC/DC' LIMIT 3", 'n